In [2]:
from utils import aStar, bfs_clusters, find_beacon

In [3]:
from PIL import Image 
import PIL 
import time

import numpy as np
import random
from collections import deque
import copy

import matplotlib.pyplot as plt
%matplotlib inline                

In [4]:
%%time
with open('tests/10', 'r') as f:
    N, MaxTips, Cost = map(int, f.readline().split())
    
    board = []
    IMG = []
    for i in range(N):
        line = f.readline().strip()
        board.append(line)
        line = line.replace("#", "1")
        line = line.replace(".", "0")
        line = list(map(int, list(line)))
        line = [[128, 128, 128] if i == 1 else [0, 0, 0] for i in line]
        IMG.append(line) 
    
    
    
    
    #ORDERS:
    raw_orders = []
    T, D = map(int, f.readline().split())
    for i in range(T):
        k = int(f.readline())
        for j in range(k):
            x1, y1, x2, y2 = map(int, f.readline().split())
            raw_orders.append([(x1 - 1, y1 - 1), (x2 - 1, y2 - 1)])
        


IMG = np.array(IMG, dtype=np.uint8)
print(MaxTips, "- чаевые") 
print(Cost, "- цена робота")
print(T, "- количество итераций")
print(D, "- Количество заказов")

#processing the map
cells = set()
for x in range(N):
    for y in range(N):
        if board[x][y] == ".":
            cells.add((x,y))  

3600 - чаевые
1048576 - цена робота
77777 - количество итераций
467760 - Количество заказов
CPU times: user 3.95 s, sys: 207 ms, total: 4.16 s
Wall time: 4.19 s


In [5]:
def draw_paths(start, clusters, cells, limit, IMG):
    global imgs
    imgs = [IMG]
    
    if start in clusters:
        ###
        img = copy.deepcopy(imgs[-1])
        img[start[0]][start[1]] = [255, 0, 0]
        imgs.append(img)
        ###
        return ["", start]
    
    used = dict()
    used[start] = [None, "s"]
    
    q = deque()
    q.append(start)
    
    counter = 0
    while q:
        x, y = q.popleft()
        ###
        img = copy.deepcopy(imgs[-1])
        img[x][y] = [255, 0, 0]
        imgs.append(img)
        ###
        
        for nx, ny, direction in [(x, y-1, "L"), (x, y+1, "R"), (x-1, y, "U"), (x+1,y, "D")]:
            next_cell = (nx, ny)
            counter += 1
            if limit and counter > limit:
                return [None, None]
            
            if next_cell in cells and next_cell not in used:
                used[next_cell] = [(x, y), direction]
                q.append(next_cell)
                
    
                if next_cell in clusters:
                    backtrack = next_cell
                    path = ""
                    while used[backtrack][1] != "s":
    
                        path += used[backtrack][1]
                        backtrack = used[backtrack][0]
                    
                    return [path[::-1], next_cell]
    print(counter)
    raise ValueError('No Path Found')

# Кластеры

In [6]:
import imageio
from PIL import Image 
import PIL 

class Animation:
    def __init__(self, IMG):
        self.imgs = [copy.deepcopy(IMG)]
        self.current = copy.deepcopy(IMG)
    
    def update(self):
        #Adding the last img to the list of images
        self.imgs.append(copy.deepcopy(self.current))
    
    def save_img(self, name):
        image_png = Image.fromarray(self.current, "RGB") 
        image_png.save(f'{name}.png')
    
    def save_gif(self, name, dur = 0.1):
        print(len(self.imgs))
        new_imgs = []
        for img in self.imgs:
            new_imgs.append(img[150:850, 150:850])
        
        imageio.mimsave(f'{name}.gif', new_imgs, duration=dur)
        
        
    def draw_points(self, points, color, updating = False):
        for x, y in points:
            self.current[x][y] = color
            if updating:
                self.update()
                
        
        
    
    
    def draw_big_point(self, point, color, updating = False):
        x, y = point
        for dx, dy in [[0, -1],[0,1],[1,0],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]:
            self.current[x + dx][y + dy] = color
            self.current[x + dx*2][y + dy*2] = color
            self.current[x + dx*3][y + dy*3] = color
      
            
        if updating:
            self.update()

In [7]:
import copy
image = copy.deepcopy(IMG) 
orders_map = [[0] * N for _ in range(N)]

for start, end in raw_orders:
    x1, y1 = start
    x2, y2 = end
    orders_map[x1][y1] += 1
    
       


raw_clusters = []
for x in range(N):
    for y in range(N):
        if orders_map[x][y] > 0:
            #if 120 < x < 670 and 300 < y < 700:
                raw_clusters.append((x,y))

random.shuffle(raw_clusters)
print(len(raw_clusters))

for x1,y1 in raw_clusters:
    for x2, y2 in raw_clusters:
        if [x1,y1] == [x2,y2]:
            continue
        if abs(x1 - x2) + abs(y1 - y2) <= 7:
            if (x2, y2) in raw_clusters:
                raw_clusters.remove((x2,y2))
        
raw_clusters = set(raw_clusters)        
print(len(raw_clusters))



#for point in raw_clusters:
#    ani.draw_big_point()

273
99


In [8]:
%%time
clusters = {cluster : {} for cluster in raw_clusters}
for point in clusters.keys():
    bfs_clusters(point, clusters, cells)

CPU times: user 26.1 s, sys: 253 ms, total: 26.4 s
Wall time: 26.5 s


In [9]:
#Creates nice gif for clusters

draw = Animation(IMG)
for point1 in clusters.keys():
    
    
    for point2 in random.sample(clusters[point1].keys(), 1):
        path = clusters[point1][point2]
        draw.draw_points(path, [255, 0, 0])
        
        draw.draw_big_point(point2, [0, 255, 255])
    draw.draw_big_point(point1, [0, 255, 255])
    
    for point3 in clusters.keys():
        draw.draw_big_point(point3, [0, 255, 255])
        if point3 == point1:
            break
    
    draw.update()
    
draw.save_gif("images/clusters")

100


In [10]:
POINTS_D = {(0, -1) : "L",
            (0, 1) : "R",
            (1, 0) : "D",
            (-1,0) : "U"}

REVERSE_DIR = {"D" : "U",
              "U" : "D",
              "R" : "L",
              "L" : "R"}


def reverse_path(path):
    #path = "DDRRL" -> "UULLR" -> "RLLUU"
    rev_path = ""
    for direction in path[::-1]:
        rev_path += REVERSE_DIR[direction]
    return rev_path

In [15]:
%%time
CENTER = (480, 555)
center_to_clusters = {point : "" for point in raw_clusters}
clusters_to_center = {}



bfs_clusters(CENTER, center_to_clusters, cells, center = True)

CPU times: user 241 ms, sys: 15.9 ms, total: 256 ms
Wall time: 258 ms


In [21]:
#Drawing  
YELLOW = [255, 255, 0]
RED = [255, 0, 0]
CYAN = [0, 255, 255]


draw = Animation(IMG)

points = list(center_to_clusters.keys())
points.sort(key = lambda point: (abs(point[0] - CENTER[0]) + abs(point[1] - CENTER[1])) )
for point in points:
    path = center_to_clusters[point]
    
    draw.draw_points(path, RED)
    draw.draw_big_point(point, CYAN)
        
    
    draw.draw_big_point(CENTER, YELLOW, updating = True)
    
draw.save_gif("images/one_center")

100


In [ ]:
#Turning points into path -> "UDDLRLDRLL"
for point in center_to_clusters.keys():
    point_path = center_to_clusters[point]
    path = ""
    for i in range(len(point_path) - 1):
        x1, y1 = point_path[i]
        x2, y2 = point_path[i + 1]
        path += POINTS_D[(x2 - x1, y2 - y1)]
    center_to_clusters[point] = reverse_path(path)
    clusters_to_center[point] = path
   

In [130]:
    
POINTS_D = {(0, -1) : "L",
            (0, 1) : "R",
            (1, 0) : "D",
            (-1,0) : "U"}
for point1 in clusters.keys():
    for point2 in clusters[point1].keys():
        point1_to_point2 = clusters[point1][point2]
        
        path = ""
        for i in range(len(point1_to_point2) - 1):
            x1, y1 = point1_to_point2[i]
            x2, y2 = point1_to_point2[i+1]
            path += POINTS_D[(x2 - x1, y2 - y1)]
        clusters[point1][point2] = path

with open('new_clusters.py', 'w') as f:
    print('clusters = ', clusters, file=f)

# Оптимизация кластеров

In [48]:
new_clusters = copy.deepcopy(clusters)
for point1 in new_clusters.keys():
	for point2 in new_clusters[point1].keys():
		if new_clusters[point1][point2] != "" and new_clusters[point2][point1] != "":
			new_clusters[point2][point1] = ""

with open('new_clusters.py', 'w') as f:
    print('clusters = ', new_clusters, file=f)

# Обработка заказов

In [131]:
%%time
LIMIT = 1000


count = 0
banned = set()
for start_order, end_order in raw_orders[:]:
    if start_order not in banned:
        _, start_cluster = find_beacon(start_order, raw_clusters, cells, limit = LIMIT)
        _, end_cluster = find_beacon(end_order, raw_clusters, cells, limit = LIMIT)
        if start_cluster and end_cluster:
            count += 1
        else:
            banned.add(start_order)
        


print(len(raw_orders))
print(count)
print(len(banned))


467760
467760
0
CPU times: user 5.23 s, sys: 3.89 ms, total: 5.23 s
Wall time: 5.26 s


In [154]:
def show_paths(R, robots):
    paths = [""] * R
    
    for robot in robots:
        path_60  = robot.get_path()
        paths[robot.idx] += path_60

    #for path in paths:
    #    print(path)
        
        

class Robot:
    def __init__(self, idx, cell):
        self.idx = idx #Его индекс
        self.path = "" #Его путь
        self.iter = 0
        self.position = cell


    def get_path(self):
        cur_path = self.path[self.iter:self.iter+60]
        self.iter += 60
        if len(cur_path) < 60:
            
            self.path = ""
            self.iter = 0
            cur_path += "S" * (60 - len(cur_path))
            chill_robots.append(self.idx)

        return cur_path

In [148]:
class Order:
    def __init__(self, start, end, iter_start):
        self.start = start
        self.end = end
        self.iter_start = iter_start #Время появления заказа

        self.beacon_to_start = None
        self.start_to_beacon = None
        self.beacon_to_end = None
        self.end_to_beacon = None
        self.cluster_start = None
        self.cluster_end = None
        
        
        self.dilevery_time = 0
        
        self.path = ""
        
        
    
    def build_path(self):
        
        
                
        self.path += self.beacon_to_start
        self.path += "T" 
        self.path += self.start_to_beacon
        
        self.path += clusters_to_center[self.cluster_start]
        self.path += center_to_clusters[self.cluster_end]
    
        self.path += self.beacon_to_end
        
        self.dilevery_time += len(self.path)
        
        
        self.path += "P"
        
        self.path += self.end_to_beacon

        
        return self.path  
    
    
    def calculate_tip(self, MaxTip, cur_iteration):
        t = 0
        t += (cur_iteration - self.iter_start) * 60
        t += self.dilevery_time
        
        return max(0, MaxTip - t)

In [149]:
print(len(raw_clusters))
print(len(orders))

101
101


In [150]:
with open('clusters_to_center.py', 'w') as f:
    print("clusters_to_center =", clusters_to_center,  file=f)
    print("center_to_clusters =", center_to_clusters, file = f)

In [162]:
%%time
#%%capture
# Получение заказов
with open('sdc_meetup_2021_tests/09', 'r') as f:
    EARNINGS = 0
    DELIVERED = 0
    
    N, MaxTips, Cost = map(int, f.readline().split())
    
    board = []
    for i in range(N):
        line = f.readline().strip()
        board.append(line)
    
    T, D = map(int, f.readline().split())
    
    
    #ROBOTS#############################
    
    R = 70 # Количество роботов
    LIMIT = 10000
    
    
    EARNINGS -= Cost * R
    print(R, flush = True)

    random_clusters = random.sample(raw_clusters, R//2) + random.sample(raw_clusters, R//2)
    
    robots = [Robot(idx, cell) for idx, cell in enumerate(random_clusters)]

    chill_robots = deque(list(range(R)))
    
    
    for robot in robots:
        x, y = robot.position
        #print(cell[0]+1, cell[1]+1, flush = True)
        

    
    #ANCHORS PART:
    cells = set()
    for x in range(N):
        for y in range(N):
            if board[x][y] == ".":
                cells.add((x,y))  

    
    #ORDERS:
    orders = {cell : deque() for cell in raw_clusters}
    
    orders_to_clusters = {}
    
    
   
    for i in range(T):
        
        
        k = int(f.readline())
        for j in range(k):
            x1, y1, x2, y2 = map(int, f.readline().split())
            order_start = (x1 - 1, y1 - 1)
            order_end = (x2 - 1, y2 - 1)

            if order_start in orders_to_clusters:
                path_start, cluster_start = orders_to_clusters[order_start]
            else:
                path_start, cluster_start = find_beacon(order_start, raw_clusters, cells, limit = LIMIT)
                orders_to_clusters[order_start] = (path_start, cluster_start)
                
            if order_end in orders_to_clusters:
                path_end, cluster_end = orders_to_clusters[order_end]
            else:
                path_end, cluster_end = find_beacon(order_end, raw_clusters, cells, limit = LIMIT)
                orders_to_clusters[order_end] = (path_end, cluster_end)
                
            
            order = Order(order_start, order_end, i + 1)
    
            order.start_to_beacon = path_start
            order.beacon_to_start = reverse_path(path_start)
            order.end_to_beacon = path_end
            order.beacon_to_end = reverse_path(path_end)
            order.cluster_start = cluster_start
            order.cluster_end = cluster_end
            
            
            
            orders[cluster_start].append(order)
        

            
        
        #Для каждого робота проверить, есть ли заказ
        #print(chill_robots)
        while chill_robots:
            robot_idx = chill_robots.popleft()
            robot = robots[robot_idx]
            
            START_POSITION = robot.position
        
        
            if orders[robot.position]:

                order = orders[robot.position].popleft()

                robot.path = order.build_path()
                robot.position = order.cluster_end


                #INFO
                DELIVERED += 1
                EARNINGS += order.calculate_tip(MaxTips, i + 1)

                #CHECKER(board, START_POSITION, robot.position, robot.path, order.start, order.end)

            else:
                #if decision(0.43):
                    #Посылаем в рандомный cell
                    random_cluster = random.sample(raw_clusters, 1)[0]


                    #robot.path = clusters[robot.position][random_cluster]
                    robot.path =  clusters_to_center[robot.position] + center_to_clusters[random_cluster]
                    robot.position = random_cluster

                    #       Поле   Начало          Конец
                    #CHECKER(board, START_POSITION, robot.position, robot.path)
                
               
 
            
        show_paths(R, robots)
        
        
print(EARNINGS)
print(DELIVERED)

70
329401301
375930
CPU times: user 14.3 s, sys: 31.7 ms, total: 14.3 s
Wall time: 14.5 s


#Before orders optimization 22.2s

x = 532 - 1
y = 553 - 1
#LDDDDLLLLLLULLULLULLULLULLULLLUULLLLLLLLLLLLLLLLLLULLLULLULU

def replace_str_index(text, index, replacement):
    return '%s%s%s'%(text[:index],replacement,text[index+1:])

board[x] = replace_str_index(board[x], y, "$")

for i in range(515, 540):
    print(board[i][520:560])

    

In [231]:
p = ""
CHECKER(board, (374, 509), (0,0), p)

[348, 511]
DRRRRRRRUURUURUUUURRUURUURUUUUULULLULLUUUULULLUULLLULULLULLU


ValueError: WALL

In [73]:
def draw_path(points, PROBLEM):
    image = copy.deepcopy(IMG)
    for x, y in points:
        image[x][y] = [255, 0, 0]
    
    x, y = PROBLEM
    image[x][y] = [0, 0, 255]
    
    new_image = Image.fromarray(image, "RGB") 
    new_image.save('ERROR.png')

ACTION_D = {"U" : (-1, 0),
            "D" : (1, 0),
            "R" : (0, 1),
            "L" : (0, -1)}

def CHECKER(board, start_position, end_position, path, order_start = None, order_end = None):
    result = None
    
    x, y = start_position
    POINTS = [[x,y]]
    PROBLEM = order_start
    for action in path:
        if action == "U":
            x -= 1
        elif action == "D":
            x += 1
        elif action == "R":
            y += 1
        elif action == "L":
            y -= 1
        POINTS.append([x,y])

        if board[x][y] == "#":
            PROBLEM = [x,y]
            result = 'WALL'
        
        elif action == "T" and order_start != (x, y):
            PROBLEM = [x,y]
            result = "NO PACKAGE"
            
        elif action == "P" and order_end != (x, y):
            PROBLEM = [x,y]
            result = "NO PLACE TO DELIVER"
    
    if (x, y) != end_position:
        PROBLEM = [x,y]
        result = "DIDN'T REACH DESTINATION"
            
    #if result is not None:
    if result:
        draw_path(POINTS, PROBLEM)
        print(PROBLEM)
        print(path)
        raise ValueError(result)

## Путь

*Кластер -> заказ_старт -> кластер -> заказ_финиш -> кластер*

# Аналитика

In [251]:
def draw_points(points):
    for point in points.values():
        for path in point.values():
            if path:
                print(path)
                for x, y in path:
                    image[x][y][0] += 100
    
    for point in points.keys():
        x, y = point
        for dx, dy in [[0, -1],[0,1],[1,0],[-1,0],[1,1],[1,-1],[-1,1],[-1,-1]]:
            image[x+dx][y+dy] = [50, 255, 255]
            image[x+dx*2][y+dy*2] = [50, 255, 255]

draw_points(clusters)

NameError: name 'clusters' is not defined

05. Среднее растояние от пункта выдачи, до пункта доставки заказа ~682
08. 531

1. Показать кластеры

2. Среднее попарное расстояние

3. Среднее расстояние кластер -> центр -> кластер

4. нарисовать и тот и другой способ. Анимация

